In [ ]:
from pathlib import Path
import sys
#src_path = str(Path.cwd().parent / "src")
#sys.path.append(src_path)

from Supply_chain_disruption_model import SimulationModel
import Supply_chain_disruption_model as scd
import Centrality as cen

import numpy as np
#from scipy.stats import poisson
#import random
import pandas as pd
import plotly.express as px

#!pip install scikit-survival

from datetime import datetime

# Simulation

Define model parameters.

In [ ]:
# p: The proportion (percentage/100) of firms that are damaged by the disruption. (default is 0.1)
# damage_level: The average amount of damage inflicted on affected firms. On average, (100*damage_level)% of the 
#     production capacity is damaged. (default is 0.2)
# margin: Specifies the margin around the given average damage level. The actual damage will lie between 
#     (damage_level - margin) and (damage_level + margin), truncated from below by zero and from above by one. 
#     (default is 0.1)
# tau: The number of days over which the inventory is restored to the target value. (default is 6)
# k: The average target inventory of a firm, specified as number of days of product use. (default is 9)
#     value=9 from paper inoue_firm-level_2019
# gamma: The recovery rate of damaged firms. (default is 0.5)
# sigma: The number of days without recovery and production in the firms damaged after the disruption. (default is 6)
# alpha: The number of days a firm tolerates a negative inventory of a supplier, before it tries to replace the supplier. 
#     (default is 2)
# u: Each firm has on average (100*u)% capacity utilization. This is used to assign a maximum possible production 
#     capacity to each firm. (default is 0.8)
# max_init_inventory: Whether firms initially have a full inventory or not. (default is True)
# fixed_target_inventory: Whether the target inventory value is fixed or determined on the previous day's realized 
#     demand. (default is True)
# nb_iter: The number of iterations (days) to run the simulation. (default is 100)
param = {"p": 0.2, "damage_level": 0.7, "margin": 0.1, "tau": 6, "k": 9, "gamma": 0.5, "sigma": 6, "alpha": 2, 
         "u": 0.8, "max_init_inventory": False, "fixed_target_inventory": True, "nb_iter": 10*365}

Generate data.

In [ ]:
# dim: number of firms
dim = 200
# nb_s: number of sectors
nb_s = 50

adj, A, C, sector = scd.generate_data(dim, nb_s)

Execute the simulation.

In [ ]:
# intialize the model
mdl = SimulationModel(A, sector, C, **param)

# time the simulation
start=datetime.now()  

# run the model
mdl.run_simulation(print_iter=False)

print(f"runtime: {datetime.now()-start}")

In [ ]:
print(f"{len(mdl.damaged_ind)} firms were damaged by the disruption, which is {100*len(mdl.damaged_ind)/dim :.1f}%.")
print(f"{len(mdl.defaults)} out of {dim} firms defaulted, which is {100*len(mdl.defaults)/dim :.1f}%.")

damaged_and_defaulted = list(set(mdl.damaged_ind) & set(mdl.defaults.keys()))

if len(mdl.damaged_ind) > 0:
    perc_damaged_and_defaulted_of_damaged = 100*len(damaged_and_defaulted)/len(mdl.damaged_ind)
    print(f"Of the damaged firms, {perc_damaged_and_defaulted_of_damaged :.1f}% defaulted, "
          f"{100 - perc_damaged_and_defaulted_of_damaged :.1f}% survived.")

if len(mdl.defaults)> 0:
    perc_damaged_and_defaulted_of_defaulted = 100*len(damaged_and_defaulted)/len(mdl.defaults)
    print(f"Of the defaulted firms, {perc_damaged_and_defaulted_of_defaulted :.1f}% had been damaged, "
          f"{100 - perc_damaged_and_defaulted_of_defaulted :.1f}% had not been damaged. \n")

    print(f"This means {100 - perc_damaged_and_defaulted_of_defaulted :.1f}% of the firms that defaulted did so due to "

          f"network propagation of the damage.")

#print(f"damaged firms that also defaulted: \n{sorted(damaged_and_defaulted)}\n")
#print(f"damaged (ind): \n{mdl.damaged_ind} \n")
#print(f"defaults (ind: iteration): \n{mdl.defaults} \n")
#print(f"defaulted firms: \n{sorted(mdl.defaults.keys())} \n")
#print(f"default times: \n{sorted(mdl.defaults.values())}")

In [ ]:
print(f"defaulted firms: {sorted(list(mdl.defaults.keys()))}\n")
print(f"damaged firms: {sorted(mdl.damaged_ind)}")

In [ ]:
# filter which firms are plotted
## all
select_firms = []  
## only those directly damaged
#select_firms = mdl.damaged_ind
## only those not directly damaged
#select_firms = [i for i in range(dim) if i not in mdl.damaged_ind]
## specific firms
#select_firms = [5, 63]

mdl.plot_capacity(relative=True, col_by_sector=False, show_leg=True, select_firms=select_firms)

In [ ]:
#mdl.plot_capacity(relative=True, col_by_sector=True, show_leg=True, select_firms=[])

In [ ]:
#mdl.plot_capacity(relative=False, col_by_sector=False)

# Time to return to equilibrium

In [ ]:
# relative production capacity data
prodcap_df = mdl.get_plot_df(relative=True, select_firms=[])
prodcap_df

In [ ]:
# max deviation from median allowed during equilibrium
mar = 0.005  # 0.001, 0.005

In [ ]:
# median production per firm
prodcap_medians = prodcap_df.groupby("firm")["prod_cap"].median().to_frame()
prodcap_medians.reset_index(inplace=True)
prodcap_medians.rename(columns={"prod_cap": "prod_cap_median"}, inplace=True)
prodcap_medians

In [ ]:
prodcap_df = prodcap_df.merge(prodcap_medians, on='firm')
prodcap_df

In [ ]:
# absolute deviation from median
prodcap_df["dev"] = abs(prodcap_df["prod_cap_median"] - prodcap_df["prod_cap"])
prodcap_df

In [ ]:
# find start time of equilibrium state
eq_ret_time = np.zeros(dim)
for i in range(dim):
    firm_df = prodcap_df.loc[prodcap_df.firm == i].copy()
    firm_df = firm_df.sort_values("x", ascending=False)
    if len(firm_df[firm_df.dev > mar]) > 0:
        eq_ret_time[i] = firm_df[firm_df.dev > mar].x.values[0] + 1
    else:
        eq_ret_time[i] = 0

# boolean indicator of whether equilibrium was reached
reached_eq = eq_ret_time != param["nb_iter"] + 1
for i in mdl.defaults.keys():
    reached_eq[i] = False
    eq_ret_time[i] = param["nb_iter"] + 1

# PN score

In [ ]:
scores = cen.PN_score(adj).reshape(1,-1)[0]

In [ ]:
damaged = np.zeros(dim).astype(bool)
damaged[mdl.damaged_ind] = True

defaulted = np.zeros(dim).astype(bool)
defaulted[list(mdl.defaults.keys())] = True

status = np.zeros(dim).astype(int).astype(str)
status[damaged] = "dam"
status[defaulted] = "def"
status[damaged*defaulted] = "dam+def"
status[(~damaged)*(~defaulted)] = "norm"

In [ ]:
fig = px.scatter(x=scores, y=eq_ret_time, color=status, symbol=reached_eq, 
                 title="Absolute values in adjacency matrix",
                 labels={'x':'PN score', 'y':'equilibrium reached time (day)', 'color':'status', 'symbol':'reached eq.'}) 
fig.show()

In [ ]:
adj_bin = np.copy(adj)
adj_bin[adj_bin > 0] = 1
adj_bin[adj_bin < 0] = -1

scores_bin = cen.PN_score(adj_bin).reshape(1,-1)[0]

In [ ]:
fig = px.scatter(x=scores_bin, y=eq_ret_time, color=status, symbol=reached_eq, 
                 title="Binary values in adjacency matrix",
                 labels={'x':'PN score', 'y':'equilibrium reached time (day)', 'color':'status', 'symbol':'reached eq.'})
fig.show()

# Survival analysis

In [ ]:
data_y = pd.DataFrame(np.transpose([reached_eq,eq_ret_time]), columns=["reached_eq","eq_time"])
data_y["reached_eq"] = data_y["reached_eq"].astype(bool)
data_y.head()

In [ ]:
data_x = pd.DataFrame(np.transpose([scores,damaged]), columns=["PN","damaged"])
data_x["damaged"] = data_x["damaged"].astype(bool)
data_x.head()

In [ ]:
import matplotlib.pyplot as plt
from sksurv.nonparametric import kaplan_meier_estimator

time, survival_prob = kaplan_meier_estimator(data_y["reached_eq"], data_y["eq_time"])
plt.step(time, survival_prob, where="post")
plt.ylabel("est. probability of survival $\hat{S}(t)$")
plt.xlabel("time $t$")
plt.show()

\textbf{Update survival analysis with defaulted + damaged status.} 

In [ ]:
for damage_status in (True, False):
    mask_damaged = data_x["damaged"] == damage_status
    time_damaged, survival_prob_damaged = kaplan_meier_estimator(
        data_y["reached_eq"][mask_damaged],
        data_y["eq_time"][mask_damaged])

    plt.step(time_damaged, survival_prob_damaged, where="post",
             label="Damaged = %s" % damage_status)

plt.ylabel("est. probability of survival $\hat{S}(t)$")
plt.xlabel("time $t$")
plt.legend(loc="best")
plt.show()

In [ ]:
from sklearn import set_config
from sksurv.linear_model import CoxPHSurvivalAnalysis

set_config(display="text")  # displays text representation of estimators

data_x_numeric = data_x.copy()
data_x_numeric["damaged"] = data_x_numeric["damaged"].astype(float)

dt = np.dtype([('reached_eq', '?'), ('eq_time', '<f8')])
data_y = np.array(list(zip(reached_eq,eq_ret_time)), dtype=dt)

estimator = CoxPHSurvivalAnalysis()
estimator.fit(data_x_numeric, data_y)

In [ ]:
pd.Series(estimator.coef_, index=data_x_numeric.columns)

In [ ]:
estimator.score(data_x_numeric, data_y)

# Build network for visualization in presentation

In [ ]:
dim = 15
nb_s = 5
adj, A, C, sector = scd.generate_data(dim, nb_s)

In [ ]:
df = pd.DataFrame(columns=["supplier","customer"])

for i in range(dim):
    customer = i
    suppliers = A[i]
    for j in range(dim):
        if suppliers[j] > 0:
            df = pd.concat([df, pd.DataFrame([[int(j), i]], columns=["supplier","customer"])])

df

In [ ]:
#df.to_csv('links.csv', index=False)

In [ ]:
df_firms = pd.DataFrame(list(range(dim)), columns=["firm"])
df_firms["sector"] = sector
df_firms

In [ ]:
#df_firms.to_csv('firms.csv', index=False)